# Test OpenAI Adapter

This notebook tests the OpenAI adapter with different configurations for code-switching analysis.


In [ ]:
# Setup: imports and environment
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Add src to path
sys.path.append('../src')

# Load API keys from .env file
load_dotenv()

# Import our adapter
from adapters.openai_adapter import OpenAIAdapter

# Verify that your key loaded correctly
print("OpenAI key:", bool(os.getenv("OPENAI_API_KEY")))


In [ ]:
# Initialize OpenAI adapter
openai_adapter = OpenAIAdapter(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",  # Cost-effective model
    temperature=0.7,
    max_tokens=500
)

print("OpenAI adapter initialized successfully!")


In [ ]:
# Test basic generation
test_prompt = "Paraphrase this sentence without changing its meaning: He finna go to the store."

response = openai_adapter.generate_response(test_prompt)
print("Response:", response)


In [ ]:
# Test with system prompt for better code-switching analysis
system_prompt = """You are a linguist analyzing code-switching behavior. 
Your task is to preserve the dialectal style and cultural context while 
paraphrasing or continuing text. Maintain the same linguistic variety 
and register as the input."""

user_prompt = "Paraphrase this AAVE sentence: He finna go to the store. You sliding?"

response = openai_adapter.generate_with_system_prompt(system_prompt, user_prompt)
print("Response with system prompt:", response)
